Workflow used for:
  1. Read phenotypic information and consolidate it with raw PGS scores from **apply_grs.ipynb**
  1. Standardize raw scores and calculate Odds Ratios between case/controls
  1. Generate visualization plots

In [1]:
# Import all necessary libraries

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
import seaborn as sns
import numpy as np
import os, glob
from sklearn import preprocessing
import statsmodels.api as sm

Step 1) Read phenotypic information and consolidate with raw PGS scores

In [2]:
os.chdir("/labs/tassimes/rodrigoguarischi/projects/sea/")

# Read phenotypes and recode sex and race attributes
sea_phenotypes = pd.read_table(
    "data_preparation_to_imputation/86679/NHLBI/SEA_Herrington/phs000349v1/p1/phenotype/phs000349.v1.pht002191.v1.p1.c1.SEA_Phase2_Subject_Phenotypes.GRU.txt",
    index_col="seaid",
    comment="#")

# Recode Sex and Race
sea_phenotypes = sea_phenotypes.replace( {
    "sex": { 1:"Male", 2:"Female" },
    "race": { 1:"White", 2:"Black" }
    } )

# Print counts by race and sex and first lines from dataframe 
print( sea_phenotypes.groupby(["race","sex"])["sex"].count() )
sea_phenotypes.head()

race   sex   
Black  Female     92
       Male      412
White  Female    128
       Male      436
Name: sex, dtype: int64


,dbGaP SubjID,sex,race,age,agex2,bmi,tf,tr,af,ar,cf,cr,rltotal,rlmean
seaid,,,,,,,,,,,,,,
pd10016,445969,Female,White,29,841,19.9,6.7,0.0,10.7,0.0,3.3,0.0,0.0,0.000000
pd10018,445970,Male,White,30,900,28.3,27.3,0.0,22.0,0.0,3.4,0.6,0.6,0.200000
pd10023,445971,Male,White,27,729,21.7,27.3,0.0,55.0,0.0,4.0,0.0,0.0,0.000000
pd10028,445972,Male,Black,31,961,29,16.0,0.0,36.8,14.8,1.7,1.0,15.8,5.266667
pd10031,445973,Male,White,28,784,21.5,20.0,0.0,16.7,0.0,0.0,0.0,0.0,0.000000


In [3]:
# Read files with raw scores of multiple PRSs

li = []
li.append(sea_phenotypes)

os.chdir("/labs/tassimes/rodrigoguarischi/projects/sea/apply_grs/")

# Dictionary with paths to score files
score_files_paths = {
    "hrc": "./michigan_hrc_20220419/*.scores.txt",
    "topmed": "./topmed_20220419/*.scores.txt"
    }

# Loop over scores files, load all into memory and save it to list object 'li'
for reference_panel_id in score_files_paths:
    
    for current_score_file in glob.glob( score_files_paths[ reference_panel_id ] ):
        
        pgs_results = pd.read_table( current_score_file, sep = "," )
        pgs_results["sample"] = pgs_results["sample"].str.split("_", expand = True)[0]
        pgs_results = pgs_results.set_index("sample")
        min_r2_used = os.path.basename( current_score_file ).split(".")[0].split("_")[-1]
        
        # Add prefix to column names. Names should match pattern <hrc|topmed>_<threshold>_<scoreid>
        pgs_results = pgs_results.add_prefix(reference_panel_id + "_" + min_r2_used + "_")

        li.append( pgs_results )

# Consolidate data into a dataframe and print first lines
sea_dataset_full = pd.concat( li, axis=1 )
sea_dataset_full.head()

,dbGaP SubjID,sex,race,age,agex2,bmi,tf,tr,af,ar,...,topmed_r05_wGRS49,topmed_r05_PGS000349,topmed_r05_PGS000018,topmed_r05_PGS000667,topmed_r05_PGS000889,topmed_r08_wGRS49,topmed_r08_PGS000349,topmed_r08_PGS000018,topmed_r08_PGS000667,topmed_r08_PGS000889
pd10016,445969,Female,White,29,841,19.9,6.7,0.0,10.7,0.0,...,-0.014892,0.876993,0.710600,2.385210,1.331384,0.500438,1.211101,0.464124,0.0,0.397157
pd10018,445970,Male,White,30,900,28.3,27.3,0.0,22.0,0.0,...,-0.119400,1.052222,1.198771,1.649050,0.948200,0.174197,0.624084,0.458905,0.0,0.373110
pd10023,445971,Male,White,27,729,21.7,27.3,0.0,55.0,0.0,...,0.137079,0.729801,0.746956,1.443510,1.508553,0.213983,0.418310,0.430099,0.0,0.814898
pd10028,445972,Male,Black,31,961,29,16.0,0.0,36.8,14.8,...,-0.301765,0.705801,1.184374,16.642935,0.264086,-0.009379,0.304889,0.743781,0.0,-0.341113
pd10031,445973,Male,White,28,784,21.5,20.0,0.0,16.7,0.0,...,-0.193940,0.980570,0.744978,3.189740,-0.079091,0.330856,0.491178,0.349139,0.0,0.201303


In [4]:
# Assign case/control classes to samples using top quartile rule splitting by sex

# Phenotype of interest
phenotype = "cr"

# Calculate thresholds for each race and sex
print( "Thresholds to be used for case/control definition:" )
thresholds = {}
for race in ["White", "Black"]:
    for sex in ["Male", "Female"]:
        
        # Define Q3 as threshold and save it in thresholds dictionary
        key = race + "_" + sex
        thresholds[ key ] = sea_dataset_full[ (( sea_dataset_full["race"]==race ) & ( sea_dataset_full["sex"]==sex ))  ][phenotype].quantile(0.75)
        
        # Print values and warnings, if needed
        note = ""
        if( thresholds[ key ] == 0 ):
            note = "(WARNING: This group has Q3 equals zero. Considering only non-zeros as CASE group!!)"
        print( " - {0} = {1:.4f} {2}".format( key, thresholds[key], note ) )

# Add a new column called Case with all values equals to False
sea_dataset_full = sea_dataset_full.assign( Case=False )

# Identify subjects above threshold to assign them to group "Case"
for i in sea_dataset_full.index:

    sex = sea_dataset_full.loc[ i, "sex"]
    race = sea_dataset_full.loc[ i, "race"]
    key = race + "_" + sex
    
    if( sea_dataset_full.loc[i, phenotype] > thresholds[ key ] ):
        sea_dataset_full.loc[i, "Case"] = True

# Print summary of Case/Controls for each subgroup
print( sea_dataset_full.groupby(["race","sex","Case"])["Case"].count() )
sea_dataset_full.head()

Thresholds to be used for case/control definition:
 - White_Male = 1.6000 
 - White_Female = 0.4250 
 - Black_Male = 1.0000 
 - Black_Female = 0.0000 (WARNING: This group has Q3 equals zero. Considering only non-zeros as CASE group!!)
race   sex     Case 
Black  Female  False     73
               True      19
       Male    False    311
               True     101
White  Female  False     96
               True      32
       Male    False    328
               True     108
Name: Case, dtype: int64


,dbGaP SubjID,sex,race,age,agex2,bmi,tf,tr,af,ar,...,topmed_r05_PGS000349,topmed_r05_PGS000018,topmed_r05_PGS000667,topmed_r05_PGS000889,topmed_r08_wGRS49,topmed_r08_PGS000349,topmed_r08_PGS000018,topmed_r08_PGS000667,topmed_r08_PGS000889,Case
pd10016,445969,Female,White,29,841,19.9,6.7,0.0,10.7,0.0,...,0.876993,0.710600,2.385210,1.331384,0.500438,1.211101,0.464124,0.0,0.397157,False
pd10018,445970,Male,White,30,900,28.3,27.3,0.0,22.0,0.0,...,1.052222,1.198771,1.649050,0.948200,0.174197,0.624084,0.458905,0.0,0.373110,False
pd10023,445971,Male,White,27,729,21.7,27.3,0.0,55.0,0.0,...,0.729801,0.746956,1.443510,1.508553,0.213983,0.418310,0.430099,0.0,0.814898,False
pd10028,445972,Male,Black,31,961,29,16.0,0.0,36.8,14.8,...,0.705801,1.184374,16.642935,0.264086,-0.009379,0.304889,0.743781,0.0,-0.341113,False
pd10031,445973,Male,White,28,784,21.5,20.0,0.0,16.7,0.0,...,0.980570,0.744978,3.189740,-0.079091,0.330856,0.491178,0.349139,0.0,0.201303,False


Step 2) **Standardize** raw scores and calculate **Odds Ratios** between case/controls

In [7]:
# Standardize raw scores and calculate OR

li = []
standardize_scores = []
logit_models_dict = {}
models_summaries = pd.DataFrame()

# Normalize all scores with mean = 0 and SD = 1
for race in ["White", "Black"]:
    
    # Subset cohort between whites and blacks to run logistic regression individually
    sea_dataset_subset = sea_dataset_full[ sea_dataset_full["race"] == race ]

    # Create a dependent variable named "Case_recoded" based on column "Case", conding it as 0 and 1 to fit glm
    sea_dataset_subset = sea_dataset_subset.assign( Case_recoded=sea_dataset_subset["Case"].replace(True, 1).replace(False, 0) )
    
    for reference_panel_id in ["topmed", "hrc"]:
        
        for threshold in ["r0", "r03", "r05", "r08"]:

            for grs in ["wGRS49","PGS000349","PGS000018", "PGS000667", "PGS000889"]:

                raw_score_name = "_".join( (reference_panel_id, threshold, grs) )
                standardize_score_name = "_".join( (race, reference_panel_id, threshold, grs) )
                standardize_scores.append(standardize_score_name)
        
                # Standardize raw scores using method scale
                sea_dataset_subset[ standardize_score_name ] = preprocessing.scale( sea_dataset_subset[ raw_score_name ] )
        
                # Fit a logistic model using standardize scores and save it on dictionary 
                logit_models_dict[ standardize_score_name ] = sm.formula.glm(
                    "Case_recoded ~ " + standardize_score_name + " + age + sex",
                    family=sm.families.Binomial(),
                    data=sea_dataset_subset).fit()

                model_summary_series = pd.Series(
                    data={
                        "odds_ratio": np.exp( logit_models_dict[ standardize_score_name ].params )[ standardize_score_name ],
                        "score_pvalue": logit_models_dict[ standardize_score_name ].pvalues[ standardize_score_name ],
                        "conf_interval_lower": np.exp( logit_models_dict[ standardize_score_name ].conf_int()[0][ standardize_score_name ] ),
                        "conf_interval_upper": np.exp( logit_models_dict[ standardize_score_name ].conf_int()[1][ standardize_score_name ] )
                    },
                    name=standardize_score_name
                )
                
                li.append(model_summary_series)
                     
# Concatenate results for all models tested and transpose object to make visualization easier 
models_summaries = pd.concat( li, axis=1 ).transpose()
models_summaries.sort_values("odds_ratio", ascending=False)

,odds_ratio,score_pvalue,conf_interval_lower,conf_interval_upper
White_topmed_r0_PGS000889,1.356405,0.003671,1.104265,1.666117
White_hrc_r05_wGRS49,1.350758,0.003299,1.105298,1.650731
White_topmed_r05_wGRS49,1.344010,0.003880,1.099649,1.642674
White_hrc_r03_PGS000889,1.342931,0.004041,1.098383,1.641927
White_hrc_r03_wGRS49,1.339253,0.004292,1.095965,1.636547
...,...,...,...,...
Black_topmed_r03_PGS000667,0.961742,0.725307,0.773675,1.195524
Black_topmed_r0_PGS000667,0.939581,0.580842,0.753112,1.172219
White_hrc_r03_PGS000667,0.911138,0.396210,0.734882,1.129669
White_hrc_r0_PGS000667,0.902715,0.331497,0.734243,1.109844


In [8]:
model_name = "White_hrc_r03_wGRS49"
print( logit_models_dict[model_name].summary() )
# print( logit_models_dict[model_name].pvalues[model_name] )
# print( np.exp( logit_models_dict[model_name].params )[ model_name ] )
# print( np.exp( logit_models_dict[model_name].conf_int().loc[[ model_name ]] ) )

                 Generalized Linear Model Regression Results                  
Dep. Variable:           Case_recoded   No. Observations:                  564
Model:                            GLM   Df Residuals:                      560
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -303.16
Date:                Mon, 25 Apr 2022   Deviance:                       606.32
Time:                        11:31:42   Pearson chi2:                     570.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -3.8150 

In [ ]:
# Plot boxplots and histograms and boxplots of raw scores spliting by sex

sns.set_style('whitegrid')

score_list = ["wGRS49_r0", "wGRS49_r03", "wGRS49_r05", "wGRS49_r08"]
# score_list = ["PGS000349_r0", "PGS000349_r03", "PGS000349_r05", "PGS000349_r08"]
# score_list = ["PGS000018_r0", "PGS000018_r03", "PGS000018_r05", "PGS000018_r08"]

fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(5*len(score_list),12))
for i in range(0, len(score_list)):
    sns.boxplot(
        x = "Case",
        y = score_list[i],
        data = sea_merged_whites,
        palette = reversed(sns.color_palette(n_colors=2)),
        width=0.4,
        fliersize=2,
        ax=axs[0,i],
    )
    sns.histplot(
        x = score_list[i],
        data = sea_merged_whites[ sea_merged_whites["sex"] == "Male" ],
        hue="Case",
        hue_order=[True, False],
        kde=True,
        ax=axs[1,i]
        ).set(title='Males (n={0})'.format( sum( sea_merged_whites["sex"] == "Male" ) ) )
    sns.histplot(
        x = score_list[i],
        data = sea_merged_whites[ sea_merged_whites["sex"] == "Female" ],
        hue="Case",
        hue_order=[True, False],
        kde=True,
        ax=axs[2,i]
    ).set(title='Females (n={0})'.format( sum( sea_merged_whites["sex"] == "Female" ) ) )

plt.subplots_adjust(top=1.25)

# # Plot only boxplots

# fig, axs = plt.subplots(ncols=len(score_list), figsize=(5*len(score_list),4))
# for i in range(0, len(score_list)):
#     sns.boxplot(
#         x = "Case",
#         y = score_list[i],
#         data = sea_merged_whites,
#         hue="sex",
#         hue_order=['Male','Female'],
#         width=0.4,
#         fliersize=2,
#         ax=axs[i],
#     )

In [ ]:
# Test variables
sns.boxplot(
    x = "Case",
    y = "age",
    data = sea_merged_whites,
    width=0.4,
    fliersize=2
)

Export dataset to CSV files

In [ ]:
# Export full dataset
sea_merged_whites.to_csv("exports/sea_whites_phenotypes_scores_full.csv")

# Export only subset of columns Tim asked
select_columns = ["dbGaP_SubjID","sex","race","age","agex2","bmi","cr","wGRS49_r05","PGS000349_r05","PGS000018_r05","wGRS49_r03","PGS000349_r03","PGS000018_r03","wGRS49_r0","PGS000349_r0","PGS000018_r0","wGRS49_r08","PGS000349_r08","PGS000018_r08", "Case"]
sea_merged_whites[select_columns].to_csv("exports/sea_whites_phenotypes_scores_column_subset.csv")

In [ ]:
import seaborn as sns

sns.scatterplot(
    x = "wGRS49_r03_scaled",
    y = "Case",
    data = sea_merged_whites
    # ,
    # alpha=0.4,
    # s=6
    )



Test logistic regression in <code>R</code>

In [ ]:
import scipy.stats as stats

# table = df.groupby(level="Cancer").sum().values

table = [[840,51663],[32,5053]]
print(table)
oddsratio, pvalue = stats.fisher_exact(table)
print("OddsR: ", oddsratio, "p-Value:", pvalue)

In [ ]:
sea_merged_whites.head()

In [ ]:
from scipy import stats

for score in ["wGRS49_r03", "wGRS49_r05", "wGRS49_r08", "PGS000018_r03", "PGS000018_r05", "PGS000018_r08"]:
    for traits in ["tf_quartile4", "tr_quartile4", "af_quartile4", "ar_quartile4", "cf_quartile4", "cr_quartile4"]:
        # score = "PGS000018_r08"
        cases = sea_merged_whites[traits]
        print( score , "\t", traits, "\t", stats.ttest_ind( sea_merged_whites[cases][score], sea_merged_whites[np.invert(cases)][score] ) )

In [ ]:
#

current_score_file = "./apply_grs/wGRS49_PGS000349_PGS000018_r0.scores.txt"

pgs_results = pd.read_table( current_score_file, sep = "," )
pgs_results["sample"] = pgs_results["sample"].str.split("_", expand = True)[0]
pgs_results = pgs_results.set_index("sample")
min_r2_used = os.path.basename( current_score_file ).split(".")[0].split("_")[-1]
pgs_results = pgs_results.add_suffix("_" + min_r2_used)
pgs_results.head()
    # score_name = os.path.basename( current_score_file ).split(".")[0]
    # pgs_results.rename( columns={ pgs_results.columns[0]:score_name }, inplace=True )
